In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [3]:
!wget $data -O lead_scoring.csv 

--2025-10-11 13:07:57--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘lead_scoring.csv’

lead_scoring.csv    100%[===================>]  78.98K  --.-KB/s    in 0.1s    

2025-10-11 13:07:58 (773 KB/s) - ‘lead_scoring.csv’ saved [80876/80876]



In [4]:
df = pd.read_csv('lead_scoring.csv')

In [5]:
df

lead_source       industry  number_of_courses_viewed  annual_income  \
0           paid_ads            NaN                         1        79450.0   
1       social_media         retail                         1        46992.0   
2             events     healthcare                         5        78796.0   
3           paid_ads         retail                         2        83843.0   
4           referral      education                         3        85012.0   
...              ...            ...                       ...            ...   
1457        referral  manufacturing                         1            NaN   
1458        referral     technology                         3        65259.0   
1459        paid_ads     technology                         1        45688.0   
1460        referral            NaN                         5        71016.0   
1461  organic_search        finance                         3        92855.0   

     employment_status       location  interaction_count  lead_score  \
0           unemployed  south_america                  4        0.94   
1             employed  south_america                  1        0.80   
2           unemployed      australia                  3        0.69   
3                  NaN      australia                  1        0.87   
4        self_employed         europe                  3        0.62   
...                ...            ...                ...         ...   
1457     self_employed  north_america                  4        0.53   
1458           student         europe                  2        0.24   
1459           student  north_america                  3        0.02   
1460     self_employed  north_america                  0        0.25   
1461           student  north_america                  3        0.41   

      converted  
0             1  
1             0  
2             1  
3             0  
4             1  
...         ...  
1457          1  
1458          1  
1459          1  
1460          1  
1461          1  

[1462 rows x 9 columns]

# Data Preparation

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [7]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'lead_score', 'interaction_count']

In [8]:
for c in categorical:
    df[c] = df[c].fillna('NA')
df['annual_income'] = df['annual_income'].fillna(0.0)    

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1462 non-null   object 
 1   industry                  1462 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1462 non-null   float64
 4   employment_status         1462 non-null   object 
 5   location                  1462 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


## Question 1
What is the most frequent observation (mode) for the column industry?

In [10]:
df['industry'].value_counts().index[0]

'retail'

## Question 2
What are the two features that have the biggest correlation?

In [11]:
df[numerical].corr()

number_of_courses_viewed  annual_income  lead_score  \
number_of_courses_viewed                  1.000000       0.009770   -0.004879   
annual_income                             0.009770       1.000000    0.015610   
lead_score                               -0.004879       0.015610    1.000000   
interaction_count                        -0.023565       0.027036    0.009888   

                          interaction_count  
number_of_courses_viewed          -0.023565  
annual_income                      0.027036  
lead_score                         0.009888  
interaction_count                  1.000000

0.009888 -0.004879 -0.023565 0.027036

In [12]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
print(len(df), len(df_full_train), len(df_test))

df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
print(len(df_train), len(df_val))

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

1462 1169 293
876 293


## Question 3
Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).
Which of these variables has the biggest mutual information score?

In [13]:
def mutual_info_churn_score(series):
    return mutual_info_score(df_full_train.converted, series)

In [14]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

lead_source          0.024562
employment_status    0.012690
industry             0.008173
location             0.001212
dtype: float64

## Question 4
- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
- To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
- model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [15]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(train_dicts)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [16]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [17]:
y_pred = model.predict(X_val)

In [18]:
accuracy = model.score(X_val, y_val)
print(f'Accuracy: {round(accuracy, 2)}')

Accuracy: 0.7


## Question 5
- Let's find the least useful feature using the feature elimination technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
- Which of following feature has the smallest difference?

In [19]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'lead_score', 'interaction_count']

In [20]:
accuracy_score = model.score(X_val, y_val)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6996587030716723


In [21]:
categorical_without_industry = ['lead_source', 'employment_status', 'location']
categorical_without_employment_status = ['lead_source', 'industry', 'location']
numerical_without_lead_source = ['number_of_courses_viewed', 'annual_income', 'interaction_count']

In [22]:
accuracy

0.6996587030716723

In [23]:
train_dicts_without_industry = df_train[categorical_without_industry + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(train_dicts_without_industry)

val_dict = df_val[categorical_without_industry + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

accuracy_score = model.score(X_val, y_val)
print(f'Accuracy: {accuracy_score}')
print('difference: ', accuracy - accuracy_score)

Accuracy: 0.6996587030716723
difference:  0.0


In [24]:
train_dicts_without_employment_status = df_train[categorical_without_employment_status + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(train_dicts_without_employment_status)

val_dict = df_val[categorical_without_employment_status + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

accuracy_score = model.score(X_val, y_val)
print(f'Accuracy: {accuracy_score}')
print('difference: ', accuracy - accuracy_score)

Accuracy: 0.7030716723549488
difference:  -0.0034129692832765013


In [25]:
train_dicts_without_lead_source = df_train[categorical + numerical_without_lead_source].to_dict(orient='records')
dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(train_dicts_without_lead_source)

val_dict = df_val[categorical + numerical_without_lead_source].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

accuracy_score = model.score(X_val, y_val)
print(f'Accuracy: {accuracy_score}')
print('difference: ', accuracy - accuracy_score)

Accuracy: 0.6996587030716723
difference:  0.0


## Question 6
- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

Note: If there are multiple options, select the smallest C.

In [26]:
C_values = [0.01, 0.1, 1, 10, 100]
results = {}

for C in C_values:
    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    
    val_dicts = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    accuracy = model.score(X_val, y_val)
    results[C] = round(accuracy, 3) 
    
    print(f'C = {C}: Accuracy = {results[C]}')

best_accuracy = max(results.values())
best_C = min([c for c, acc in results.items() if acc == best_accuracy])
print(f'\nBest C: {best_C} with accuracy {best_accuracy}')

C = 0.01: Accuracy = 0.7
C = 0.1: Accuracy = 0.7
C = 1: Accuracy = 0.7
C = 10: Accuracy = 0.7
C = 100: Accuracy = 0.7

Best C: 0.01 with accuracy 0.7
